In [1]:
import dask.dataframe as dd
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
unames = ['userId', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('users.dat', sep='::', 
                      header=None, names=unames, engine='python')
rnames = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movieId', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python')

In [2]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings = movies_with_ratings.dropna()

dataset_0 = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset = dd.from_pandas(dataset_0, npartitions=20)

In [9]:
dataset

,uid,iid,rating
npartitions=20,,,
0,float64,object,float64
50015,...,...,...
...,...,...,...
950382,...,...,...
1000385,...,...,...


In [10]:
from surprise.model_selection.validation import cross_validate

In [11]:
reader = Reader(rating_scale=(dataset_0.rating.min(), dataset_0.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [12]:
#n = 1
#dataset = dataset_0.sample(frac=1)[:int(n*len(dataset_0))].reset_index(drop=True)

#int(n*len(dataset_0))

### ^количество строк массива с данными

In [13]:
algo = KNNBaseline(k=40, min_k=20, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, ['rmse'], n_jobs = -1, verbose = True)

MemoryError: Unable to allocate 278. MiB for an array with shape (6040, 6040) and data type float64

### Лучший результат который удалось получить Mean_RMSE = 0.8771